<a href="https://colab.research.google.com/github/lucasfreire01/Cancer_pele/blob/main/Cancerpele.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
!pip install cloudpathlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [131]:
!pip install pyvips

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [132]:
!apt list libvips

Listing... Done


In [133]:
!apt-get update
!apt-get install libvips

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 222 kB in 1s (165 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libvips42' instead of 'libvips'
libv

In [134]:
!pip install --upgrade fancyimpute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [135]:
!pip install fsspec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [136]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [137]:
import warnings
warnings.filterwarnings('ignore')

In [138]:
import pandas as pd
import numpy as np
from cloudpathlib import S3Path
import matplotlib.pyplot as plt
import pyvips
from PIL import Image
import tensorflow as tf

In [139]:
Image.MAX_IMAGE_PIXELS = None

In [140]:
train = pd.read_csv('/content/drive/MyDrive/Cancer_pele/train_metadata_pPa7YD3.csv')
train

,filename,age,sex,body_site,melanoma_history,breslow,ulceration,resolution,tif_cksum,tif_size,us_tif_url,eu_tif_url,as_tif_url
0,1u4lhlqb.tif,[32:34[,2,thigh,YES,<0.8,NO,0.264384,3028450373,747151312,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
1,rqumqnfp.tif,[46:48[,1,trunc,NO,[1 : 2[,NO,0.264384,1294832049,591027450,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
2,bu5xt1xm.tif,[64:66[,2,face,NO,<0.8,NO,0.264384,774102360,465947458,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
3,dibvu7wk.tif,[62:64[,2,forearm,NaN,[2 : 4[,YES,0.226490,515827065,568174704,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
4,qsza4coh.tif,[90:92[,2,face,NO,[1 : 2[,NO,0.226490,1541795099,1042691978,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,n7jd638y.tif,[68:70[,1,head/neck,NaN,[1 : 2[,NO,0.250000,1949851255,136729436,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
1338,8wg601oe.tif,[66:68[,1,head/neck,NaN,<0.8,NaN,0.250000,1344207528,1603484314,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
1339,1ejfd01b.tif,[80:82[,1,trunk,NaN,>=4,YES,0.250000,4292211076,1423564688,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...
1340,2ztrjp20.tif,[42:44[,2,trunk,NaN,<0.8,NaN,0.250000,2949914817,336489430,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...


In [141]:
train_labels = pd.read_csv('/content/drive/MyDrive/Cancer_pele/train_labels.csv')
train_labels

,filename,relapse
0,1u4lhlqb.tif,0
1,rqumqnfp.tif,0
2,bu5xt1xm.tif,0
3,dibvu7wk.tif,0
4,qsza4coh.tif,0
...,...,...
1337,n7jd638y.tif,0
1338,8wg601oe.tif,0
1339,1ejfd01b.tif,0
1340,2ztrjp20.tif,0


In [142]:
train_labels.relapse.value_counts()

0    1129
1     213
Name: relapse, dtype: int64

In [143]:
train['melanoma_history'].value_counts()

NO     585
YES     57
Name: melanoma_history, dtype: int64

In [144]:
train['melanoma_history'].fillna('YES', inplace=True)

In [145]:
train['melanoma_history'].value_counts()

YES    757
NO     585
Name: melanoma_history, dtype: int64

In [146]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder(drop="first", sparse_output=False)
one.fit(np.array(train['melanoma_history']).reshape(-1,1))
one.get_feature_names_out()

array(['x0_YES'], dtype=object)

In [147]:
train['age_int'] = train['age'].str.extract(r'(\d+)').astype(int)

In [148]:
train

,filename,age,sex,body_site,melanoma_history,breslow,ulceration,resolution,tif_cksum,tif_size,us_tif_url,eu_tif_url,as_tif_url,age_int
0,1u4lhlqb.tif,[32:34[,2,thigh,YES,<0.8,NO,0.264384,3028450373,747151312,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,32
1,rqumqnfp.tif,[46:48[,1,trunc,NO,[1 : 2[,NO,0.264384,1294832049,591027450,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,46
2,bu5xt1xm.tif,[64:66[,2,face,NO,<0.8,NO,0.264384,774102360,465947458,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,64
3,dibvu7wk.tif,[62:64[,2,forearm,YES,[2 : 4[,YES,0.226490,515827065,568174704,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,62
4,qsza4coh.tif,[90:92[,2,face,NO,[1 : 2[,NO,0.226490,1541795099,1042691978,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,n7jd638y.tif,[68:70[,1,head/neck,YES,[1 : 2[,NO,0.250000,1949851255,136729436,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,68
1338,8wg601oe.tif,[66:68[,1,head/neck,YES,<0.8,NaN,0.250000,1344207528,1603484314,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,66
1339,1ejfd01b.tif,[80:82[,1,trunk,YES,>=4,YES,0.250000,4292211076,1423564688,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,80
1340,2ztrjp20.tif,[42:44[,2,trunk,YES,<0.8,NaN,0.250000,2949914817,336489430,s3://drivendata-competition-visiomel-public-us...,s3://drivendata-competition-visiomel-public-eu...,s3://drivendata-competition-visiomel-public-as...,42


In [149]:
from sklearn.preprocessing import OneHotEncoder
one = OneHotEncoder()
one_hot = pd.get_dummies(train['melanoma_history'])
one_hot

,NO,YES
0,0,1
1,1,0
2,1,0
3,0,1
4,1,0
...,...,...
1337,0,1
1338,0,1
1339,0,1
1340,0,1


In [150]:
train_new = train[['filename', 'age', 'sex', 'body_site', 'melanoma_history',
       'resolution']]

In [151]:
train_new

,filename,age,sex,body_site,melanoma_history,resolution
0,1u4lhlqb.tif,[32:34[,2,thigh,YES,0.264384
1,rqumqnfp.tif,[46:48[,1,trunc,NO,0.264384
2,bu5xt1xm.tif,[64:66[,2,face,NO,0.264384
3,dibvu7wk.tif,[62:64[,2,forearm,YES,0.226490
4,qsza4coh.tif,[90:92[,2,face,NO,0.226490
...,...,...,...,...,...,...
1337,n7jd638y.tif,[68:70[,1,head/neck,YES,0.250000
1338,8wg601oe.tif,[66:68[,1,head/neck,YES,0.250000
1339,1ejfd01b.tif,[80:82[,1,trunk,YES,0.250000
1340,2ztrjp20.tif,[42:44[,2,trunk,YES,0.250000


In [152]:
train_new.drop('filename', axis=1, inplace=True)

In [153]:
train_new['age'] = train['age_int']
train_new.drop('melanoma_history', axis=1, inplace=True)
train_new['melanoma_NO'] = one_hot['NO']
train_new['melanoma_YES'] = one_hot['YES']
train_new

,age,sex,body_site,resolution,melanoma_NO,melanoma_YES
0,32,2,thigh,0.264384,0,1
1,46,1,trunc,0.264384,1,0
2,64,2,face,0.264384,1,0
3,62,2,forearm,0.226490,0,1
4,90,2,face,0.226490,1,0
...,...,...,...,...,...,...
1337,68,1,head/neck,0.250000,0,1
1338,66,1,head/neck,0.250000,0,1
1339,80,1,trunk,0.250000,0,1
1340,42,2,trunk,0.250000,0,1


In [154]:
train_new.isnull().sum()

age              0
sex              0
body_site       18
resolution       0
melanoma_NO      0
melanoma_YES     0
dtype: int64

In [155]:
mode = train_new['body_site'].mode().values[0]
train_new['body_site'].fillna(mode, inplace=True)

In [156]:
train_new.isnull().sum()

age             0
sex             0
body_site       0
resolution      0
melanoma_NO     0
melanoma_YES    0
dtype: int64

In [157]:
train_new['resolution'] = train_new['resolution'].round(2)

In [158]:
train_new

,age,sex,body_site,resolution,melanoma_NO,melanoma_YES
0,32,2,thigh,0.26,0,1
1,46,1,trunc,0.26,1,0
2,64,2,face,0.26,1,0
3,62,2,forearm,0.23,0,1
4,90,2,face,0.23,1,0
...,...,...,...,...,...,...
1337,68,1,head/neck,0.25,0,1
1338,66,1,head/neck,0.25,0,1
1339,80,1,trunk,0.25,0,1
1340,42,2,trunk,0.25,0,1


In [159]:
numeric = train_new[['age', 'sex', 'resolution', 'melanoma_NO', 'melanoma_YES']]
numeric

,age,sex,resolution,melanoma_NO,melanoma_YES
0,32,2,0.26,0,1
1,46,1,0.26,1,0
2,64,2,0.26,1,0
3,62,2,0.23,0,1
4,90,2,0.23,1,0
...,...,...,...,...,...
1337,68,1,0.25,0,1
1338,66,1,0.25,0,1
1339,80,1,0.25,0,1
1340,42,2,0.25,0,1


In [160]:
train_new['body_site'].value_counts()

trunc                  324
leg                    163
arm                    149
face                   134
trunk                  132
lower limb/hip         114
head/neck               71
upper limb/shoulder     70
thigh                   58
forearm                 26
foot                    23
hand/foot/nail          21
sole                    13
neck                    12
toe                      7
hand                     7
finger                   6
seat                     6
scalp                    4
nail                     2
Name: body_site, dtype: int64

In [161]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
body = label_encoder.fit_transform(train_new['body_site'])
body = tf.keras.utils.to_categorical(body)

In [162]:
test_new = train_labels['relapse']

In [170]:
encoded_bd = pd.DataFrame(one.fit_transform(train_new[['body_site']]).toarray())
categories = train_new['body_site'].unique()
encoded_bd.columns = categories

In [171]:
train_new = pd.concat([train_new, encoded_bd], axis=1)

In [173]:
train_new = train_new.drop('body_site', axis=1)

In [174]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_new = pd.DataFrame(scaler.fit_transform(train_new), columns=train_new.columns)

In [177]:
test_new = test_new.astype('float32')
test_new1 = tf.convert_to_tensor(test_new)
train_new1 = tf.convert_to_tensor(train_new)

In [191]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_new,test_new, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [192]:
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
X_val = tf.convert_to_tensor(X_val)

y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)
y_val = tf.convert_to_tensor(y_val)

In [194]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
model = Sequential()
model.add(Dense(64, input_dim=25, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
early_stop = EarlyStopping(monitor="val_loss", patience=5)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stop])


loss, accuracy = model.evaluate(X_test, y_test, batch_size=32)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/50
27/27 [==============================] - 1s 11ms/step - loss: 0.5554 - accuracy: 0.7739 - val_loss: 0.4800 - val_accuracy: 0.8186
Epoch 2/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4660 - accuracy: 0.8380 - val_loss: 0.4579 - val_accuracy: 0.8186
Epoch 3/50
27/27 [==============================] - 0s 3ms/step - loss: 0.4606 - accuracy: 0.8427 - val_loss: 0.4551 - val_accuracy: 0.8186
Epoch 4/50
27/27 [==============================] - 0s 4ms/step - loss: 0.4569 - accuracy: 0.8415 - val_loss: 0.4527 - val_accuracy: 0.8186
Epoch 5/50
27/27 [==============================] - 0s 4ms/step - loss: 0.4442 - accuracy: 0.8403 - val_loss: 0.4516 - val_accuracy: 0.8186
Epoch 6/50
27/27 [==============================] - 0s 4ms/step - loss: 0.4354 - accuracy: 0.8415 - val_loss: 0.4507 - val_accuracy: 0.8186
Epoch 7/50
27/27 [==============================] - 0s 4ms/step - loss: 0.4393 - accuracy: 0.8415 - val_loss: 0.4502 - val_accuracy: 0.8186
Epoch 8/50
27/27 [=

In [190]:
from keras.models import Sequential
from keras.layers import Dense


model = Sequential()
model.add(Dense(64, input_dim=10, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
score = model.evaluate(x_test, y_test, batch_size=32)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10


ValueError: ignored